In [1]:
# imports
%load_ext autoreload
%autoreload 2
from dataset_wrapper import find_and_load_datasets
from classifier_wrapper import SKLearnClassifierWrapper
from preprocessing_wrapper import PreprocessingWrapper
from logger import Logger
from features import FeatureExtraction


from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# other models, preprocessing, scalers etc.

In [2]:
# paths, constants
root = "../dataset-private/" # path to the datasets
validation_split=0.1

In [3]:
experiment_name = "testing1"
model_save_path = f"./models/model_{experiment_name}.pkl"

## Loading, init

In [4]:
loaders = find_and_load_datasets(root,batch_size=1000)
print("Found datasets:", list(loaders.keys()))

""" results = sample_n_from_each_dataset(loaders,n=3)
for ds_name, info in results.items():
    print(f"Dataset: {ds_name}  (file used: {info['file']})  samples: {len(info['samples'])}")
    display(info['df']) """

for name, loader in loaders.items():
    print(f"{name}: {len(loader)} samples")

Found datasets: ['008-zeek-mixed', '009-zeek-malicious', '010-zeek-mixed', '011-zeek-mixed', '012-zeek-mixed', '013-zeek-mixed', '014-zeek-malicious', '015-zeek-malicious']
008-zeek-mixed: 5603 samples
009-zeek-malicious: 79401 samples
010-zeek-mixed: 5310 samples
011-zeek-mixed: 8722 samples
012-zeek-mixed: 3810 samples
013-zeek-mixed: 26241 samples
014-zeek-malicious: 26738 samples
015-zeek-malicious: 197722 samples


## Pipeline

In [5]:
#feature processing
feature_extraction = FeatureExtraction()

#preprocessing
scaler = StandardScaler() 
preprocessor = PreprocessingWrapper()
preprocessor.add_step("scaler", scaler)
#other steps? pca?

#classifier
model = SGDClassifier(loss='hinge', penalty='l2') 
classifier = SKLearnClassifierWrapper(model)

# reporting
logger = Logger(experiment_name = experiment_name,overwrite=True)

In [ ]:

# datasets we have from before - select one (or a list) to train on!
# main loop
    # call batch from dataset
    # process features
    # preprocessing (scaling)
    # train on model with validation, logger for metrics!
    # save model after the whole dataset is done
    # reporting, metrics, plots, etc.
dataset_to_train_on = '008-zeek-mixed'
loader = loaders[dataset_to_train_on]

for i in range(loader.batches()):
    print(f"Processing batch {i+1}")
    batch = loader.next_batch()
    X, y = feature_extraction.process_batch(batch)
    sum_labeled_flows = len(y)
    X_train, X_val, y_gt_train, y_gt_val = train_test_split(X, y, test_size=validation_split, random_state=42)
    
    preprocessor.partial_fit(X_train)
    X_train_processed = preprocessor.transform(X_train)
    y_pred_train = classifier.partial_fit(X_train_processed, y)

    # predict on validation set
    X_val_processed = preprocessor.transform(X_val)
    y_pred_val = classifier.predict(X_val_processed)
    logger.save_training_results(
         y_pred_train, y_gt_train, y_pred_val, y_gt_val, sum_labeled_flows
    )

Processing batch 1
Batch size: 1000
Error in FeatureExtraction.process_batch():
Traceback (most recent call last):
  File "/home/svobojan/StratosphereLinuxIPS/pipeline_ml_training/features.py", line 83, in process_batch
    df = self._process_features(df)
  File "/home/svobojan/StratosphereLinuxIPS/pipeline_ml_training/features.py", line 183, in _process_features
    df = df.drop(columns=[c for c in self._builtin_drop + self.columns_to_discard if c in df.columns], errors="ignore")
AttributeError: 'FeatureExtraction' object has no attribute '_builtin_drop'



ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# final test?
# on unrelated dataset?